<a href="https://colab.research.google.com/github/cjh2001525/Intelligent-Weather-Analysis-Advisory-System/blob/main/starter_notebook%20by%20Junhan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [ ]:
# === Install Dependencies (Colab/Jupyter only) ===
# Use this cell ONLY if you see ModuleNotFoundError when importing

# Weather + HTTP
!pip install requests

# Visualization
!pip install matplotlib

# Input validation for menu interface
!pip install pyinputplus

# Optional helper packages (we are not using them by default)
# !pip install fetch-my-weather
# !pip install hands-on-ai


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyinputplus: filename=pyinputplus-0.2.12-py3-none-any.whl size=11348 sha256=871871f47f960230f6de29bdeb25fc1e66c3973fa22c035c24b4ac50e9584339
  Stored in directory: /root/.cache/pip/wheels/85/81/c1/325c8199c5c4de8a0b9af900ff9cd8bd69853e3d9419493512


In [ ]:
# === Imports and Configuration ===
import os
import re
import requests
import matplotlib.pyplot as plt
from datetime import datetime, timezone
from typing import Dict, Any

# Try to import pyinputplus (menu library)
try:
    import pyinputplus as pyip
    _HAS_PYIP = True
except ImportError:
    _HAS_PYIP = False

# Optional: AI tools (commented out unless needed for logging AI interactions)
# from fetch_my_weather import get_weather
# from hands_on_ai.chat import get_response

# Global config
WTTR_URL = "https://wttr.in/{location}?format=j1"
REQUEST_TIMEOUT = 12
DEFAULT_FORECAST_DAYS = 5
UNITS = "metric"  # Celsius, km/h

# Matplotlib defaults
plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["axes.grid"] = True

# === Utility Functions ===
def clamp_forecast_days(days: int) -> int:
    """Clamp forecast days to [1, 5]."""
    try:
        d = int(days)
    except Exception:
        d = DEFAULT_FORECAST_DAYS
    return max(1, min(5, d))

def clean_location(location: str) -> str:
    """Trim whitespace and collapse multiple spaces."""
    if not isinstance(location, str):
        return ""
    return re.sub(r"\s+", " ", location.strip())

def http_get_json(url: str) -> Dict[str, Any]:
    """HTTP GET JSON with basic error wrapping."""
    try:
        r = requests.get(url, timeout=REQUEST_TIMEOUT)
        r.raise_for_status()
        return {"ok": True, "data": r.json()}
    except requests.RequestException as e:
        return {"ok": False, "error": str(e)}

def now_iso() -> str:
    """UTC timestamp string for returned payloads/logging."""
    return datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")




## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
# === Setup and Configuration ===
import os
import re
import requests
import matplotlib.pyplot as plt
from datetime import datetime, timezone
from typing import Dict, Any

# Try to import pyinputplus (menu library)
try:
    import pyinputplus as pyip
    _HAS_PYIP = True
except ImportError:
    _HAS_PYIP = False

# Optional: AI tools (commented out unless needed for logging AI interactions)
# from fetch_my_weather import get_weather
# from hands_on_ai.chat import get_response

# Global config for weather API
WTTR_URL = "https://wttr.in/{location}?format=j1"
REQUEST_TIMEOUT = 12
DEFAULT_FORECAST_DAYS = 5
UNITS = "metric"  # Celsius, km/h

# Matplotlib defaults
plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["axes.grid"] = True

# === Utility Functions ===
def clamp_forecast_days(days: int) -> int:
    """Clamp forecast days to [1, 5]."""
    try:
        d = int(days)
    except Exception:
        d = DEFAULT_FORECAST_DAYS
    return max(1, min(5, d))

def clean_location(location: str) -> str:
    """Trim whitespace and collapse multiple spaces."""
    if not isinstance(location, str):
        return ""
    return re.sub(r"\s+", " ", location.strip())

def http_get_json(url: str) -> Dict[str, Any]:
    """HTTP GET JSON with basic error wrapping."""
    try:
        r = requests.get(url, timeout=REQUEST_TIMEOUT)
        r.raise_for_status()
        return {"ok": True, "data": r.json()}
    except requests.RequestException as e:
        return {"ok": False, "error": str(e)}

def now_iso() -> str:
    """UTC timestamp string for returned payloads/logging."""
    return datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")



## 🌤️ Weather Data Functions

In [ ]:
def get_weather_data(location: str, forecast_days: int = 5) -> Dict[str, Any]:
    """
    Retrieve weather data for a specified location using the wttr.in API.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    # Ensure valid forecast days
    forecast_days = clamp_forecast_days(forecast_days)
    location = clean_location(location)

    # Build request
    url = WTTR_URL.format(location=location)
    result = http_get_json(url)
    if not result["ok"]:
        return {"ok": False, "error": result.get("error", "Unknown error")}

    data = result["data"]

    # Extract current weather
    current = data["current_condition"][0]
    current_weather = {
        "temperature": int(current["temp_C"]),
        "humidity": int(current["humidity"]),
        "condition": current["weatherDesc"][0]["value"],
        "wind_speed": int(current["windspeedKmph"]),
        "wind_direction": current["winddir16Point"],
        "precipitation": float(current["precipMM"]),
    }

    # Extract forecast (limit by forecast_days)
    forecast_list = []
    for day in data["weather"][:forecast_days]:
        forecast_list.append({
            "date": day["date"],
            "min_temp": int(day["mintempC"]),
            "max_temp": int(day["maxtempC"]),
            "condition": day["hourly"][4]["weatherDesc"][0]["value"],  # midday snapshot
            "precipitation_chance": int(day["hourly"][4]["chanceofrain"]),
        })

    return {
        "ok": True,
        "location": location,
        "units": UNITS,
        "timestamp": now_iso(),
        "current": current_weather,
        "forecast": forecast_list,
    }


## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data (from get_weather_data)
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object.
        Otherwise, displays the visualisation in the notebook and returns None.
    """
    # Basic validation
    if not isinstance(weather_data, dict) or weather_data.get("ok") is False:
        print(f"Error: {weather_data.get('error','No valid weather data.') if isinstance(weather_data, dict) else 'No valid weather data.'}")
        return None

    forecast = weather_data.get("forecast", [])
    if not forecast:
        print("No forecast data available.")
        return None

    # Prepare data
    dates = [day.get("date", "") for day in forecast]
    min_temps = [day.get("min_temp") for day in forecast]
    max_temps = [day.get("max_temp") for day in forecast]

    # Plot
    fig, ax = plt.subplots()
    ax.plot(dates, min_temps, marker="o", label="Min Temp (°C)")
    ax.plot(dates, max_temps, marker="o", label="Max Temp (°C)")
    ax.set_title(f"Temperature Trend for {weather_data.get('location','')}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Temperature (°C)")
    ax.legend()

    if output_type == "figure":
        return fig
    else:
        plt.show()
        return None




In [ ]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data (from get_weather_data)
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object.
        Otherwise, displays the visualisation in the notebook and returns None.
    """
    # Basic validation
    if not isinstance(weather_data, dict) or weather_data.get("ok") is False:
        print(f"Error: {weather_data.get('error','No valid weather data.') if isinstance(weather_data, dict) else 'No valid weather data.'}")
        return None

    forecast = weather_data.get("forecast", [])
    if not forecast:
        print("No forecast data available.")
        return None

    # Prepare data
    dates = [day.get("date", "") for day in forecast]
    precipitation_chances = [day.get("precipitation_chance", 0) for day in forecast]

    # Plot
    fig, ax = plt.subplots()
    ax.bar(dates, precipitation_chances)
    ax.set_title(f"Precipitation Chances for {weather_data.get('location','')}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Chance of Rain (%)")
    ax.set_ylim(0, 100)

    if output_type == "figure":
        return fig
    else:
        plt.show()
        return None


## 🤖 Natural Language Processing

In [ ]:
# === Natural Language Processing ===
import re
from datetime import datetime

def parse_weather_question(question: str) -> dict:
    """
    Parse a natural language weather question.

    Returns a dict like:
      {
        "topic": "rain" | "temperature" | "wind" | "humidity" | "general",
        "day": int (0=today, 1=tomorrow, etc.),
        "raw": original_question
      }
    """
    if not isinstance(question, str) or not question.strip():
        return {"topic": "general", "day": 0, "raw": question}

    q = question.strip().lower()

    # Topic detection
    topic = "general"
    if any(w in q for w in ["rain", "umbrella", "wet", "shower", "storm"]):
        topic = "rain"
    elif any(w in q for w in ["temp", "temperature", "hot", "cold", "warm", "chilly"]):
        topic = "temperature"
    elif any(w in q for w in ["wind", "windy", "breeze", "gale"]):
        topic = "wind"
    elif any(w in q for w in ["humid", "humidity", "moist"]):
        topic = "humidity"

    # Day detection
    day_offset = None
    if "today" in q:
        day_offset = 0
    elif "tomorrow" in q:
        day_offset = 1
    else:
        weekdays = ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]
        for i, name in enumerate(weekdays):
            if name in q:
                today_idx = datetime.today().weekday()   # Monday=0
                target_idx = i
                day_offset = (target_idx - today_idx) % 7
                break

    if day_offset is None:
        day_offset = 0

    return {"topic": topic, "day": int(day_offset), "raw": question}


def generate_weather_response(parsed_question: dict, weather_data: dict) -> str:
    """
    Generate a natural language response based on parsed question and weather data.
    """
    if not isinstance(parsed_question, dict):
        return "Sorry, I could not understand the question."
    if not isinstance(weather_data, dict) or (weather_data.get("ok") is False):
        return "Sorry, weather data is not available right now."

    topic = parsed_question.get("topic", "general")
    day = int(parsed_question.get("day", 0))

    location = weather_data.get("location", "your location")
    current = weather_data.get("current", {})
    forecast = weather_data.get("forecast", [])

    # Fallback if forecast is missing
    if not forecast:
        temp = current.get("temperature")
        cond = current.get("condition", "unavailable")
        if temp is None:
            return f"Current conditions for {location} are {cond}."
        return f"Right now in {location} it is {temp}°C and {cond}."

    # Clamp day index
    if day < 0:
        day = 0
    if day >= len(forecast):
        day = len(forecast) - 1

    day_data = forecast[day]
    day_label = "today" if day == 0 else ("tomorrow" if day == 1 else f"on {day_data.get('date','that day')}")

    if topic == "rain":
        chance = day_data.get("precipitation_chance")
        if chance is None:
            return f"Rain probability {day_label} in {location} is unavailable."
        verdict = "likely" if chance >= 50 else "unlikely"
        return f"Rain is {verdict} {day_label} in {location} (chance {chance}%)."

    elif topic == "temperature":
        tmin = day_data.get("min_temp")
        tmax = day_data.get("max_temp")
        cond = day_data.get("condition", "unknown conditions")
        if tmin is None or tmax is None:
            return f"Temperature details {day_label} in {location} are unavailable."
        return f"{day_label.capitalize()} in {location}: {cond}, {tmin}–{tmax}°C."

    elif topic == "wind":
        ws = current.get("wind_speed")
        wd = current.get("wind_direction", "")
        if ws is None:
            return f"Wind details are not available right now for {location}."
        return f"Current wind in {location} is about {ws} km/h {wd}. Daily wind forecast is not available."

    elif topic == "humidity":
        h = current.get("humidity")
        if h is None:
            return f"Humidity is not available right now for {location}."
        return f"Current humidity in {location} is {h}%."

    # General / fallback
    tnow = current.get("temperature")
    cnow = current.get("condition", "unknown conditions")
    if tnow is None:
        return f"Current conditions for {location}: {cnow}."
    return f"Right now in {location} it is {tnow}°C and {cnow}. {day_label.capitalize()} looks {day_data.get('condition','unclear')} with {day_data.get('min_temp','?')}–{day_data.get('max_temp','?')}°C."


## 🧭 User Interface

In [ ]:
# === User Interface ===

# Uses _HAS_PYIP from Setup cell (pyinputplus available or not)

def prompt_location() -> str:
    """
    Ask the user for a location and return a cleaned string.
    """
    raw = input("Enter a city or location: ")
    loc = clean_location(raw)
    while not loc:
        print("Please enter a non-empty location.")
        raw = input("Enter a city or location: ")
        loc = clean_location(raw)
    return loc

def _menu_with_pyip(choices):
    """Menu using pyinputplus if available."""
    return pyip.inputMenu(choices, numbered=True)

def _menu_with_input(choices):
    """Simple numbered menu implemented with input()."""
    print("\n=== WeatherWise Menu ===")
    for i, c in enumerate(choices, start=1):
        print(f"{i}. {c}")
    choice = input("Select an option (1–{0}): ".format(len(choices))).strip()
    while choice not in [str(i) for i in range(1, len(choices) + 1)]:
        print("Invalid choice, please try again.")
        choice = input("Select an option (1–{0}): ".format(len(choices))).strip()
    return choices[int(choice) - 1]

def display_menu() -> str:
    """
    Show the app menu and return the selected option string.
    """
    choices = [
        "Current Weather",
        "5-Day Forecast",
        "Temperature Chart",
        "Precipitation Chart",
        "Ask a Question",
        "Exit",
    ]
    if '_HAS_PYIP' in globals() and _HAS_PYIP:
        return _menu_with_pyip(choices)
    return _menu_with_input(choices)

def show_current_weather(weather_data: dict) -> None:
    """
    Pretty-print current weather from the normalised schema.
    """
    if not isinstance(weather_data, dict) or weather_data.get("ok") is False:
        print(f"Error: {weather_data.get('error','Unknown error') if isinstance(weather_data, dict) else 'Unknown error'}")
        return
    cur = weather_data.get("current", {}) or {}
    print("\n=== Current Weather ===")
    print(f"Location: {weather_data.get('location','N/A')}")
    print(f"Condition: {cur.get('condition','N/A')}")
    print(f"Temperature: {cur.get('temperature','N/A')} °C")
    print(f"Humidity: {cur.get('humidity','N/A')} %")
    print(f"Wind: {cur.get('wind_speed','N/A')} km/h {cur.get('wind_direction','')}")
    print(f"Precipitation: {cur.get('precipitation','N/A')} mm")

def show_forecast(weather_data: dict) -> None:
    """
    Print a simple table of the multi-day forecast.
    """
    if not isinstance(weather_data, dict) or weather_data.get("ok") is False:
        print(f"Error: {weather_data.get('error','Unknown error') if isinstance(weather_data, dict) else 'Unknown error'}")
        return
    fc = weather_data.get("forecast", []) or []
    if not fc:
        print("No forecast data available.")
        return
    print("\n=== 5-Day Forecast ===")
    for d in fc:
        print(f"{d.get('date','?')}: {d.get('condition','?')} | "
              f"{d.get('min_temp','?')}–{d.get('max_temp','?')} °C | "
              f"Rain chance: {d.get('precipitation_chance','?')}%")

def ui_loop(weather_data: dict) -> None:
    """
    Main UI loop for menu-driven interaction.
    Expects a valid weather_data dict produced by get_weather_data().
    """
    if not isinstance(weather_data, dict) or weather_data.get("ok") is False:
        print(f"Error: {weather_data.get('error','Unknown error') if isinstance(weather_data, dict) else 'Unknown error'}")
        return

    while True:
        choice = display_menu()

        if choice == "Current Weather":
            show_current_weather(weather_data)

        elif choice == "5-Day Forecast":
            show_forecast(weather_data)

        elif choice == "Temperature Chart":
            create_temperature_visualisation(weather_data, output_type="display")

        elif choice == "Precipitation Chart":
            create_precipitation_visualisation(weather_data, output_type="display")

        elif choice == "Ask a Question":
            q = input("Ask a weather question (e.g., 'Will it rain tomorrow?'): ")
            parsed = parse_weather_question(q)
            answer = generate_weather_response(parsed, weather_data)
            print("\n=== Answer ===")
            print(answer)

        elif choice == "Exit":
            print("Goodbye!")
            break


## 🧩 Main Application Logic

In [ ]:
# === Main Application Logic ===

def main() -> None:
    """
    Entry point for the WeatherWise app.
    Prompts for a location, fetches weather data, and starts the UI loop.
    """
    print("=== WeatherWise ===")
    location = prompt_location()

    print(f"Fetching weather for: {location} ...")
    data = get_weather_data(location, forecast_days=5)

    if not isinstance(data, dict) or data.get("ok") is False:
        print(f"Error retrieving weather data: {data.get('error', 'Unknown error') if isinstance(data, dict) else 'Unknown error'}")
        return

    # Start interactive menu loop
    ui_loop(data)


# Uncomment this line to launch the app when running the notebook
# main()


## 🧪 Testing and Examples

In [ ]:
# This cell demonstrates the end-to-end behaviour of the app.

def _print_section(title: str):
    print("\n" + "=" * 8 + f" {title} " + "=" * 8)

# 1) Fetch data for a demo location
_print_section("Fetch Weather Data")
try:
    demo_location = "Perth"
    data = get_weather_data(demo_location, forecast_days=5)
    if not isinstance(data, dict) or data.get("ok") is False:
        raise RuntimeError(data.get("error", "Unknown error") if isinstance(data, dict) else "Invalid data")
    print(f"OK: Retrieved weather for {data.get('location')} at {data.get('timestamp', 'N/A')}")
except Exception as e:
    print(f"FAILED to fetch weather data: {e}")

# 2) Show current weather and 5-day forecast (text output)
_print_section("Current Weather")
show_current_weather(data)

_print_section("5-Day Forecast")
show_forecast(data)

# 3) Create visualisations (temperature trend + precipitation chance)
_print_section("Temperature Chart")
create_temperature_visualisation(data, output_type="display")

_print_section("Precipitation Chart")
create_precipitation_visualisation(data, output_type="display")

# 4) NLP Q&A examples
_print_section("NLP Q&A")
examples = [
    "Will it rain tomorrow in Perth?",
    "Is it going to be hot today?",
    "What is the wind like?",
    "How humid is it?",
    "Weather on Friday?"
]
for q in examples:
    parsed = parse_weather_question(q)
    ans = generate_weather_response(parsed, data)
    print(f"Q: {q}\nA: {ans}\n")

# 5) Optional: interactive UI loop (commented by default)
# Uncomment the two lines below to try the interactive app after the above demos.
# _print_section("Interactive Menu")
# ui_loop(data)


Testing and Examples 2

In [ ]:
# === Extended Tests: Multi-city Max Temperature Comparison ===
cities = ["Perth", "Sydney"]
series = {}

for c in cities:
    d = get_weather_data(c, 5)
    if isinstance(d, dict) and d.get("ok"):
        series[c] = {
            "dates": [day["date"] for day in d["forecast"]],
            "max":   [day["max_temp"] for day in d["forecast"]],
        }
    else:
        print(f"Skipping {c}: {d.get('error','error')}")

# Plot on a single chart (no custom colors)
if series:
    import matplotlib.pyplot as plt
    plt.figure()
    for city, s in series.items():
        plt.plot(s["dates"], s["max"], marker="o", label=f"{city} Max (°C)")
    plt.title("Max Temperature Comparison")
    plt.xlabel("Date")
    plt.ylabel("Max Temperature (°C)")
    plt.legend()
    plt.show()


In [ ]:
# === Extended Tests: NLP Weekday Parsing & Robustness ===
d = get_weather_data("Perth", 5)
if isinstance(d, dict) and d.get("ok"):
    questions = [
        "Weather on Monday?",
        "Will it rain on Thursday?",
        "Is it hot tomorrow?",
        "How about today?",
        "   ",                     # empty-ish input
        123,                      # non-string input
        "WIND details please",    # wind keyword
    ]
    for q in questions:
        parsed = parse_weather_question(q) if isinstance(q, str) else parse_weather_question(str(q))
        ans = generate_weather_response(parsed, d)
        print(f"\nQ: {q}\nParsed: {parsed}\nA: {ans}")
else:
    print("Cannot run NLP tests: weather data unavailable.")


In [ ]:
# === Extended Tests: Edge Cases & Invalid Inputs ===
def try_fetch(loc, days):
    print(f"\n-- get_weather_data('{loc}', {days}) --")
    data = get_weather_data(loc, days)
    if not isinstance(data, dict) or data.get("ok") is False:
        print("Result: ERROR ->", data.get("error", "Unknown error") if isinstance(data, dict) else "Invalid payload")
    else:
        fc = data.get("forecast", [])
        print(f"Result: OK | location={data.get('location')} | days={len(fc)} | clamped_days={days}")

# forecast_days clamping tests
try_fetch("Perth", 0)    # should clamp to 1
try_fetch("Perth", 10)   # should clamp to 5
try_fetch("Perth", 3)    # normal

# invalid location tests
try_fetch("", 3)               # empty
try_fetch("   ", 3)            # whitespace
try_fetch("ThisIsNotACity", 3) # likely invalid


## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.